# Script to screen works for possible Commons upload

## Configuration section

In [25]:
# Import modules
import json
import csv
import math
import datetime
from time import sleep
import requests
import re # regex
# Pandas for data frame management
import pandas as pd
# Fuzzy string matching
from fuzzywuzzy import fuzz # fuzzy logic matching

# Note: setting the index to be the Q ID requires that qid has a unique value for each row. This should be the case.
works_metadata = pd.read_csv('../works_multiprop.csv', na_filter=False, dtype = str)
works_metadata.set_index('qid', inplace=True)
image_dimensions = pd.read_csv('image_dimensions.csv', na_filter=False, dtype = str)
works_classification = pd.read_csv('../../gallery_buchanan/works_classification.csv', na_filter=False, dtype = str)
works_classification.set_index('qid', inplace=True)
works_ip_status = pd.read_csv('../items_status_abbrev.csv', na_filter=False, dtype = str)
works_ip_status.set_index('qid', inplace=True)

# For testing purposes, just use the first few rows of the works metadata
test_rows = 5
works_metadata = works_metadata.head(test_rows).copy()


In [26]:
works_metadata.head(2)

,label_en,description_en,inventory_number_uuid,inventory_number,inventory_number_collection,inventory_number_ref1_hash,inventory_number_ref1_statedIn,inventory_number_ref1_referenceUrl,inventory_number_ref1_retrieved_nodeId,inventory_number_ref1_retrieved_val,...,collection_ref1_retrieved_val,collection_ref1_retrieved_prec,location_uuid,location,location_ref1_hash,location_ref1_statedIn,location_ref1_referenceUrl,location_ref1_retrieved_nodeId,location_ref1_retrieved_val,location_ref1_retrieved_prec
qid,,,,,,,,,,,,,,,,,,,,,
Q102305506,The Christ of the Breadlines,print by Fritz Eichenberg,60965535-6F8D-4CB7-B702-D6A84E576DA3,2017.001.062,Q18563658,d1f8238d0b4bf5fadeb75381feceacfdd7f12af5,,https://library.artstor.org/#/asset/26760365,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,2020-11-30T00:00:00Z,11,87C848D7-1589-4F67-9DBE-12FE22CFF157,Q18563658,d1f8238d0b4bf5fadeb75381feceacfdd7f12af5,,https://library.artstor.org/#/asset/26760365,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,11
Q102315563,Seated female funerary figure,ceramics by Artist Unknown,9438F518-B646-4E12-AADD-D5FF50DF3346,1994.413,Q18563658,7f7046f91b3d0ddd7d26cffcc9f1054db778fafa,,https://library.artstor.org/#/asset/26757434,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,2020-11-30T00:00:00Z,11,83482B6C-AC70-46B8-AC76-2E65F1C42351,Q18563658,7f7046f91b3d0ddd7d26cffcc9f1054db778fafa,,https://library.artstor.org/#/asset/26757434,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,11


In [14]:
image_dimensions.head(2)

,name,accession,kilobytes,height,width,subdir,extension
0,1956.058.tif,1956.058,197147,6540,10288,1956,tif
1,1956.028.jpg,1956.028,71,428,312,1956,jpg


In [22]:
works_classification.head(2)

,dimension,type,label
qid,,,
Q103296133,2D,artists book,(Shinzo) Ukiyo E Ruiko; two volumes
Q102961187,2D,artists book,1 Sekunde artist's book


In [24]:
works_ip_status.head(2)

,accession_number,label_en,description_en,creator,instance_of,inception_val,inception_earliest_date_val,inception_latest_date_val,height_val,width_val,thickness_val,diameter_val,style_period,rights,media_url,status
qid,,,,,,,,,,,,,,,,
Q102976613,1970.067,Veduta della Piazza di Monte Cavallo (View of ...,print by Giovanni Battista Piranesi,Q316307,Q11060274,1908-01-01T00:00:00Z,1870-01-01T00:00:00Z,1945-01-01T00:00:00Z,18.5,27.75,,,,COPYRIGHT NOT EVALUATED,https://forum.jstor.org/assets/26753692/repres...,artist died before copyright cutoff
Q102976639,1970.062,Veduta Interna dell' Atrio del Portico di Otta...,print by Giovanni Battista Piranesi,Q316307,Q11060274,1908-01-01T00:00:00Z,1870-01-01T00:00:00Z,1945-01-01T00:00:00Z,16.5,21.75,,,,NO KNOWN COPYRIGHT,https://forum.jstor.org/assets/26753687/repres...,artist died before copyright cutoff


In [34]:
print(works_classification.loc['Q102305506'])

dimension                       ambiguous
type                                print
label        The Christ of the Breadlines
Name: Q102305506, dtype: object


In [46]:
# The row index is the Q ID and is a string. The work object is the data in the row and is a Pandas series
# The items in the row series can be referred to by their labels, which are the column headers, e.g. work['label_en']
for index, work in works_metadata.iterrows():
    
    # Screen for 2 dimensional works
    try: # Error trap for the case where the index isn't found in the lookup table.
        # Find the row whose index matches the Q ID of the work, then the item by name within the series (dimension)
        # Note: this method of location works because the Q ID index is unique for each row in the lookup table.
        dimension = works_classification.loc[index, 'dimension']
    except:
        dimension = ''
    if dimension != '2D': # skip this work if not 2D
        continue
        
    
    print(work['label_en'], dimension)

'I Hate You For Hitting My Mother,’ Minneapolis 2D
